<a href="https://colab.research.google.com/github/Cyrill98/Emotion-Recognition-ML/blob/main/FER2013.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from zipfile import ZipFile

filename = "archive (1).zip"

with ZipFile(filename,'r') as zip:
  zip.extractall()
  print("Done")

Done


In [ ]:
import numpy as np
import cv2
from keras.models import Sequential
from keras.layers import Dense,Dropout,Flatten
from keras.layers import Conv2D
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from keras.layers import MaxPooling2D


In [ ]:
train_gen = ImageDataGenerator(rescale = 1./255)
val_gen = ImageDataGenerator(rescale = 1./255)

train_set = train_gen.flow_from_directory(
    'train', 
    target_size = (48,48), 
    batch_size = 64,
    color_mode = 'rgb',
    class_mode = 'categorical')

test_set = val_gen.flow_from_directory(
    'test', 
    target_size = (48,48), 
    batch_size = 64,
    color_mode = 'rgb',
    class_mode = 'categorical')

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [ ]:
import tensorflow as tf

In [ ]:
base_model = tf.keras.applications.ResNet50(input_shape=(48,48,3), include_top=False,weights='imagenet')

In [ ]:
model = Sequential()
model.add(base_model)
model.add(Conv2D(32, kernel_size=(3,3), activation='relu', padding='same',input_shape=(48,48,3),data_format='channels_last'))
model.add(Conv2D(64, kernel_size=(3,3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Conv2D(128,kernel_size=(3,3),padding='same',activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), padding='same'))
model.add(Dropout(0.25))
model.add(Conv2D(128,kernel_size=(3,3),padding='same',activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), padding='same'))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(1024,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(7,activation='softmax'))

In [ ]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 2, 2, 2048)        23587712  
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 2, 2, 32)          589856    
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 2, 2, 64)          18496     
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 1, 1, 64)          0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 1, 1, 64)          0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 1, 1, 128)         73856     
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 1, 1, 128)        

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer=Adam(
    lr=0.0001, decay=1e-6,amsgrad='True'),metrics=['accuracy'])

model_info = model.fit_generator(train_set,steps_per_epoch=449,epochs=100,validation_data=test_set,validation_steps=112)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")
/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/100
449/449 [==============================] - 4000s 9s/step - loss: 1.6411 - accuracy: 0.3186 - val_loss: 3.8584 - val_accuracy: 0.1724
Epoch 2/100
449/449 [==============================] - 3988s 9s/step - loss: 1.3409 - accuracy: 0.5078 - val_loss: 1.4546 - val_accuracy: 0.4805
Epoch 3/100
105/449 [======>.......................] - ETA: 49:33 - loss: 1.1041 - accuracy: 0.5893

In [ ]:
model.save('emotion.h5')

In [ ]:
from keras.models import load_model
model = load_model('model.h5')